# Description

---

### Introduction
There are numerous ways data can be messy or corrupted. Sometimes data is missing. Sometimes data entry errors occur. And sometimes anomalous data presents itself. These are just a few of the ways data can be messy. Proper munging is required to procure optimal results. Therefore, to challenge students, I am simulating several of these data issues that are common in the wild. The specifics can be found in the next section.

### Corruption Process
There are three key corruption processes that will be utilized. First, outliers related to age will be introduced. It should be unclear to someone who doesn't know the generative process whether these are data entry errors or true outliers. Secondly, obvious data entry errors are introduced. For example, we know GPA should be in the range of 1.0 to 4.0 but there is a case where GPA = 4.21. Lastly, missing values and an incorrect data type are introduced. I removed certain values that were close to the mean for a given feature. The assumption is that novice data scientists will impute with the mean, hence the choice. See below for details.

---

# Libraries

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from __future__ import print_function

# Versions

In [2]:
!python --version

Python 2.7.13 :: Anaconda 4.4.0 (x86_64)


In [3]:
items = [("Numpy", np), ("Pandas", pd), ("Matplotlib", matplotlib), ("Seaborn", sns)]
for item in items:
    print(item[0] + " version: " + str(item[1].__version__))

Numpy version: 1.13.0
Pandas version: 0.20.1
Matplotlib version: 2.0.2
Seaborn version: 0.7.1


# Get Data

In [4]:
# Read from disk
data = pd.read_hdf('/Users/davidziganto/Repositories/Synthetic_Dataset_Generation/data/py27/simulated_raw_training_data_py27.h5', 'table')

# DF Details

In [5]:
data.head()

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
0,2,33,0,6,2.87,7,0,11,1,9,1
1,2,52,0,7,3.19,32,1,7,1,12,1
2,2,20,1,4,2.13,0,1,18,0,0,1
3,0,31,0,0,2.73,2,0,0,0,0,0
4,3,42,1,4,2.60,22,1,17,0,0,0


In [6]:
data.describe()

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,1.521750,39.259750,0.500250,3.533250,2.891528,10.524750,0.487500,9.730250,0.507750,5.474250,0.698750
std,1.117515,12.434566,0.500062,2.276552,0.490791,9.534181,0.499906,6.032212,0.500002,6.784272,0.458858
min,0.000000,18.000000,0.000000,0.000000,1.310000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.000000,0.000000,2.000000,2.550000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000
50%,2.000000,39.000000,1.000000,4.000000,2.900000,8.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,3.000000,50.000000,1.000000,6.000000,3.240000,16.000000,1.000000,15.000000,1.000000,11.000000,1.000000
max,3.000000,60.000000,1.000000,7.000000,4.000000,42.000000,1.000000,20.000000,1.000000,20.000000,1.000000


# Corruption Process

Three key areas will be addressed:
1. Adding outliers
2. Simulating data entry errors
3. Generating missing & inf values

First, please note that I considered replacing many of the numeric features with strings. For example, changing 0 and 1 in *gender* to female and male. The code to do this is trivial and is found below. The problem is that assessing students who choose different mapping schemes from string to number makes assessment nearly impossible. Therefore, I decided to forgo this method.

In [7]:
# Considered changing numbers to text but would make scoring difficult bc how numbers are coded matters, 
# so decided to avoid altogether. Will include dictionary instead.

# Change degree to strings
#data.degree.replace(range(4), ['NB','B','M','P'], inplace=True)
# Change gender to F/M
#data.gender.replace(range(2), ['F','M'], inplace=True)
# Change major to strings
#data.major.replace(range(8), ['AN','BI','BS','CH','EN','JO','MA','PS'], inplace=True)
# Change bootcamp to No/Yes
#data.bootcamp.replace(range(2), ['No','Yes'], inplace=True)
# Change blog to No/Yes
#data.blog.replace(range(2), ['No','Yes'], inplace=True)
# Change hirec to No/Yes
#data.hired.replace(range(2), ['No','Yes'], inplace=True)

### 1. Add Outliers

In [8]:
# Note: these changes do NOT impact the models but are here for future improvements

np.random.seed(25)

age_outlier_idx = np.where(data['age']==39)[0][np.random.choice(range(len(data[data['age']==39])), size=3)]
for i, _ in data[data['age']==39].iterrows():
    if i in age_outlier_idx:
        data.loc[i, 'age'] = int(np.random.choice(a=range(120,161), size=1))

### 2. Simulate Data Entry Error

In [9]:
# Note: these changes definitely affect models bc values are not possible!

np.random.seed(42)
input_error_idx = np.random.randint(low=0, high=len(data), size=3)

# GPA range is [1.0, 4.0] so add one that's too high (e.g. 4.21)
data.loc[input_error_idx[0], 'gpa'] = 4.21

# blogger var is [0, 1] but add value of 2
data.loc[input_error_idx[1], 'blogger'] = 2
data.loc[input_error_idx[2], 'blogger'] = 2

In [10]:
data.describe()

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,1.521750,39.332000,0.500250,3.533250,2.891935,10.524750,0.487500,9.730250,0.508250,5.474250,0.698750
std,1.117515,12.710848,0.500062,2.276552,0.491209,9.534181,0.499906,6.032212,0.500994,6.784272,0.458858
min,0.000000,18.000000,0.000000,0.000000,1.310000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.000000,0.000000,2.000000,2.550000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000
50%,2.000000,39.000000,1.000000,4.000000,2.900000,8.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,3.000000,50.000000,1.000000,6.000000,3.240000,16.000000,1.000000,15.000000,1.000000,11.000000,1.000000
max,3.000000,143.000000,1.000000,7.000000,4.210000,42.000000,1.000000,20.000000,2.000000,20.000000,1.000000


### 3. Generate Missing Values & Inf

In [11]:
np.random.seed(199)

# simulate missing data
data.loc[np.where(data['experience'] == 18)[0][0], 'experience'] = np.nan
data.loc[np.where(data['github'] == 10)[0][0], 'github'] = np.nan

# Randomly select index to corrupt
nonsense_idx = np.random.randint(low=0, high=len(data), size=1)

# add 'inf' data type
data.loc[nonsense_idx, 'articles'] = np.inf

#### Proof of Corruption: Nulls

In [12]:
data.isnull().sum()

degree        0
age           0
gender        0
major         0
gpa           0
experience    1
bootcamp      0
github        1
blogger       0
articles      0
hired         0
dtype: int64

In [13]:
data.iloc[np.where(data.isnull())[0]]

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
24,1,56,0,6,2.70,31.0,0,NaN,1,7.0,0
98,1,57,0,5,2.99,NaN,1,12.0,0,0.0,1


#### Proof of Corruption: Inf

In [14]:
np.isinf(data.articles).sum()

1

In [15]:
data.iloc[np.where(data==np.inf)[0]]

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
3492,0,39,1,2,2.04,10.0,1,2.0,0,inf,1


#### Proof of Corruption: Big Picture

In [16]:
data.describe()

,degree,age,gender,major,gpa,experience,bootcamp,github,blogger,articles,hired
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,3999.000000,4000.000000,3999.000000,4000.000000,4000.000000,4000.000000
mean,1.521750,39.332000,0.500250,3.533250,2.891935,10.522881,0.487500,9.730183,0.508250,inf,0.698750
std,1.117515,12.710848,0.500062,2.276552,0.491209,9.534640,0.499906,6.032965,0.500994,NaN,0.458858
min,0.000000,18.000000,0.000000,0.000000,1.310000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.000000,0.000000,2.000000,2.550000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000
50%,2.000000,39.000000,1.000000,4.000000,2.900000,8.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,3.000000,50.000000,1.000000,6.000000,3.240000,16.000000,1.000000,15.000000,1.000000,11.000000,1.000000
max,3.000000,143.000000,1.000000,7.000000,4.210000,42.000000,1.000000,20.000000,2.000000,inf,1.000000


# Write To Disk

In [17]:
# Write to disk
data.to_hdf('/Users/davidziganto/Repositories/Synthetic_Dataset_Generation/data/py27/simulated_messy_training_data_py27.h5',
            'table',
            mode='w',
            append=True, 
            complevel=9, 
            complib='blosc',
            fletcher32=True)